In [5]:
import os
import cv2
import json
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
FILE_PATH = os.path.join(os.getcwd(), '../test/jackal_evaluation_data')
#Check if file exists
MASK_PATH = os.path.join(FILE_PATH,'masks')
if not os.path.exists(MASK_PATH):
    os.makedirs(MASK_PATH)
TRANSFORM_FILE = "transforms.json"
#Open transforms.json
with open(os.path.join(FILE_PATH,TRANSFORM_FILE)) as f:
    transforms = json.load(f)

In [7]:
new_frames = []

cnt = 0
for f in transforms["frames"]:
    #Get Frame path
    frame_path = os.path.join(FILE_PATH,f["file_path"])
    
    #Open Frame
    frame = cv2.imread(frame_path)
    #Find edge
    edge = cv2.Canny(frame,0,25)
    edge_points = np.where(edge==255)
    edge_points = np.array(edge_points)
    
    #Create mask with polygons from edge
    mask = np.zeros((frame.shape[0],frame.shape[1]),dtype=np.uint8)
    pts = edge_points.T.reshape(-1,1,2)
    #Reverse order of points
    pts = pts[:,:,::-1]
    cv2.fillPoly(mask, pts = [pts], color=(255,255,255))
    mask = cv2.dilate(mask,np.ones((3,3),np.uint8),iterations = 1)
    mask = cv2.erode(mask,np.ones((3,3),np.uint8),iterations = 1)
    
    #Save mask
    mask_path = os.path.join( "masks","{}.png".format(cnt))
    cv2.imwrite(os.path.join(FILE_PATH,mask_path),mask)

    f_copy = f.copy()
    f_copy["mask_path"] = mask_path
    new_frames.append(f_copy)

    cnt += 1


In [8]:
transforms["frames"] = new_frames
with open(os.path.join(FILE_PATH, TRANSFORM_FILE), 'w') as outfile:
    json.dump(transforms, outfile, indent=4)